In [83]:
from flask import Flask
app = Flask(__name__)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello():
    return "<h1>Hello Python</h1>"

# spring의 main과 유사한
# applicationrunner와 같은느낌
if __name__=="__main__":
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [42]:
# GET /oauth/authorize?client_id={beba58f2adeff7b7dbbc58a195847624}&redirect_uri={https://localhost.com}&response_type=code HTTP/1.1
# Host: kauth.kakao.com
#     kauth.kakao.com/oauth/authorize?client_id={beba58f2adeff7b7dbbc58a195847624}&redirect_uri={https://localhost.com}&response_type=code
#  이런식으로 요청

In [13]:
import requests

url = "https://kauth.kakao.com/oauth/token"

#data = {
#    "grant_type" : "authorization_code",
#    "client_id" : "REST API 키,
#    "redirect_uri" : "https://localhost.com",
#    "code" : "발급받은 토큰값"
#}

data = {
    "grant_type" : "authorization_code",
    "client_id" : "rest",
    "redirect_uri" : "https://localhost.com",
    "code" : "받은토큰값"
}

response = requests.post(url, data=data)

if response.status_code != 200:
    print("Error!")
else:
    tokens = response.json()
    print(tokens)
    
#     토큰시간 만료때문에 먼저 코드작성후에 kauth.kakao.com에 요청을 보낸다음 토큰 넣고 요청보낼것

{'access_token': 'B3gr9leXD207TdRJOLnbdnXju151YlgFGqNZago9dZsAAAF8Mc2wnw', 'token_type': 'bearer', 'refresh_token': 'BTRaomukRiJKAKxgup8bkFEN3h55ejVjoiJ9pwo9dZsAAAF8Mc2wng', 'expires_in': 21599, 'scope': 'account_email talk_message profile_nickname', 'refresh_token_expires_in': 5183999}


In [14]:
import json
import requests
import datetime
import os

# Kakao Token 저장 파일
KAKAO_TOKEN_SAVED_FILE = "res/kakao_msg/kakao_token.json"


#저장
def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)
        
# 읽기
def load_token(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

# Access Token 갱신을 위한 Refresh
def update_tokens(app_key, filename):
    tokens = load_token(filename)
    
    url = "https://kauth.kakao.com/oauth/token"
    
    data = {
        "grant_type" : "refresh_token",
        "client_id" : app_key,
        "refresh_toekn" : tokens['refresh_token']
    }
    
    response = requests.post(url, data=data)
    
    if response.status_code != 200:
        print("Error!")
        tokens = None
    else:
        print(response.json())
        
        # 기존 내용 백업
        now = datetime.datetime.now().strfile("%Y%m%d_%H%M%S")
        backup_filename = filename + now
        os.rename(filename, backup_filename)
        
        #새롭게 업데이트된 토큰 저장
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)
        
    return tokens

# token 저장
save_tokens(KAKAO_TOKEN_SAVED_FILE, tokens)

In [15]:
# kakaoTal message 보내기
# 스프링이 클라이언트가 되서 파이썬에 요청을 보내는 방식
import json
import requests

# 토큰 정보 활용
tokens = load_token(KAKAO_TOKEN_SAVED_FILE)

# 메시지 보내기 URL
url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

# requests parameter 설정
headers = {
    "Authorization" : "Bearer " + tokens['access_token']
}

data = {
    "template_object" : json.dumps({
        "object_type" : "text",
        "text" : "카카오톡 첫 메시지를 발송합니다.",
        "link" : {
            # 이동할 링크 정보
            "web_url" : "www.naver.com"
        }
    })
}

# 메시지 전송
response = requests.post(url, headers=headers, data=data)
print(response.status_code)

if response.status_code != 200:
    print("Error")
else:
    print("카톡 메시지가 간드아")

200
카톡 메시지가 간드아


In [89]:
from flask import Flask, url_for
app = Flask(__name__)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello():
    return "<h1>Hello Python</h1>"

# 가변인자 받기 특정아이디로 카톡 받기

@app.route("/member/<id>")
def get_member(id):
    return 'member  : ' + id

if __name__ == "__main__":
    with app.test_request_context():
        print(url_for('hello'))
        print(url_for('get_member', id='ree'))

/
/member/ree


In [ ]:
from flask import Flask, url_for, request
app = Flask(__name__)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello():
    return "<h1>Hello Python</h1>"

# 가변인자 받기 특정아이디로 카톡 받기

@app.route("/member/<id>")
def get_member(id):
    return 'member  : ' + id

# post와 get을 가능 
# 물론 구분해서도 사용하지만 이렇게 한 메서드로 다 받을수있다.
@app.route("/test", methods=['POST', 'GET'])
def chk_get_post(enity=None):
    err = None
    
    if request.method == "POST":
        id = request.form['id']
        email = request.form['email']
        
        if not id and not eamil:
            return add_entity(request.form)
    else:
        err = "Invaild!"
        
    return err
        
if __name__=="__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 19:22:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:22:48] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:23:05] "GET /test HTTP/1.1" 200 -


In [ ]:
# vue 연동시 cors 설정
# ananconda -> pip install flask_cors 인스톨
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

app = Flask(__name__)
CORS(app)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello():
    return "<h1>Hello Python</h1>"

# 가변인자 받기 특정아이디로 카톡 받기

@app.route("/member/<id>")
def get_member(id):
    return 'member  : ' + id

# post와 get을 가능
@app.route("/login", methods=['POST', 'GET'])
def chk_get_post(entity=None):
    err = None
    
    if request.method == "POST":
        data = request.get_json()
        print(data)
#         print("POST 처리")
#         userId = request.form['userId']
#         pw = request.form['password']

        
#         return "userId = " + userId + ", password = " + pw
        return jsonify(data)
    else:
        print("GET 처리")
        err = "Invaild!"
        
    return err
        
if __name__=="__main__":
    app.run()
    
#     위처럼 정보를 받아서 카카오 알림으로 전송하는방법으로 응용

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Oct/2021 15:48:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2021 15:48:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Oct/2021 15:48:18] "OPTIONS /login HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2021 15:48:18] "POST /login HTTP/1.1" 200 -


{'userId': 'test', 'password': 'test'}
